# HHL implementation in qiskit
### Victoria Zhang and Lucy Jiao

## HHL on simulators

In [52]:
from qiskit import QuantumRegister, QuantumCircuit
import numpy as np
from qiskit.providers.aer import QasmSimulator
from qiskit.visualization import plot_histogram

t = 1  # This is not optimal; As an exercise, set this to the
       # value that will get the best results. See section 8 for solution.

NUM_QUBITS = 4  # Total number of qubits
nb = 1  # Number of qubits representing the solution
nl = 2  # Number of qubits representing the eigenvalues

theta = np.pi/4  # Angle defining |b>

a = 1.5  # Matrix diagonal
b = 0.5  # Matrix off-diagonal

# Initialize the quantum and classical registers
qr = QuantumRegister(NUM_QUBITS)

# Create a Quantum Circuit
qc = QuantumCircuit(qr)

qrb = qr[0:nb]
qrl = qr[nb:nb+nl]
qra = qr[nb+nl:nb+nl+1]

# State preparation.
qc.ry(2*theta, qrb[0])

# QPE with e^{iAt}
for qu in qrl:
    qc.h(qu)

qc.p(a*t, qrl[0])
qc.p(a*t*2, qrl[1])

qc.u(b*t, -np.pi/2, np.pi/2, qrb[0])


# Controlled e^{iAt} on \lambda_{1}:
params=b*t

qc.p(np.pi/2,qrb[0])
qc.cx(qrl[0],qrb[0])
qc.ry(params,qrb[0])
qc.cx(qrl[0],qrb[0])
qc.ry(-params,qrb[0])
qc.p(3*np.pi/2,qrb[0])

# Controlled e^{2iAt} on \lambda_{2}:
params = b*t*2

qc.p(np.pi/2,qrb[0])
qc.cx(qrl[1],qrb[0])
qc.ry(params,qrb[0])
qc.cx(qrl[1],qrb[0])
qc.ry(-params,qrb[0])
qc.p(3*np.pi/2,qrb[0])

# Inverse QFT
qc.h(qrl[1])
qc.rz(-np.pi/4,qrl[1])
qc.cx(qrl[0],qrl[1])
qc.rz(np.pi/4,qrl[1])
qc.cx(qrl[0],qrl[1])
qc.rz(-np.pi/4,qrl[0])
qc.h(qrl[0])

# Eigenvalue rotation
t1=(-np.pi +np.pi/3 - 2*np.arcsin(1/3))/4
t2=(-np.pi -np.pi/3 + 2*np.arcsin(1/3))/4
t3=(np.pi -np.pi/3 - 2*np.arcsin(1/3))/4
t4=(np.pi +np.pi/3 + 2*np.arcsin(1/3))/4

qc.cx(qrl[1],qra[0])
qc.ry(t1,qra[0])
qc.cx(qrl[0],qra[0])
qc.ry(t2,qra[0])
qc.cx(qrl[1],qra[0])
qc.ry(t3,qra[0])
qc.cx(qrl[0],qra[0])
qc.ry(t4,qra[0])
qc.measure_all()

print(f"Depth: {qc.depth()}")
print(f"CNOTS: {qc.count_ops()['cx']}")
qc.draw(fold=-1)

Depth: 26
CNOTS: 10


┌─────────┐┌─────────────────┐┌────────┐┌───┐┌─────────┐┌───┐┌──────────┐┌─────────┐┌────────┐┌───┐┌───────┐┌───┐┌────────┐┌─────────┐                                                                                                               ░ ┌─┐         
 q74_0: ┤ Ry(π/2) ├┤ U(0.5,-π/2,π/2) ├┤ P(π/2) ├┤ X ├┤ Ry(0.5) ├┤ X ├┤ Ry(-0.5) ├┤ P(3π/2) ├┤ P(π/2) ├┤ X ├┤ Ry(1) ├┤ X ├┤ Ry(-1) ├┤ P(3π/2) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────░─┤M├─────────
        └──┬───┬──┘└────┬────────┬───┘└────────┘└─┬─┘└─────────┘└─┬─┘└──────────┘└─────────┘└────────┘└─┬─┘└───────┘└─┬─┘└────────┘└─────────┘                      ┌──────────┐     ┌───┐                                                                   ░ └╥┘┌─┐      
 q74_1: ───┤ H ├────────┤ P(1.5) ├────────────────■───────────────■─────────────────────────────────────┼─────────────┼──────────────────────────■───────────────■──┤ Rz(-π/4) ├─────┤ H ├────────■────────────────────────────────────────■─────────────────░──╫─┤M├──────
           ├───┤        └┬──────┬┘                                                                      │             │    ┌───┐   ┌──────────┐┌─┴─┐┌─────────┐┌─┴─┐└──────────┘     └───┘        │                                        │                 ░  ║ └╥┘┌─┐   
 q74_2: ───┤ H ├─────────┤ P(3) ├───────────────────────────────────────────────────────────────────────■─────────────■────┤ H ├───┤ Rz(-π/4) ├┤ X ├┤ Rz(π/4) ├┤ X ├─────■────────────────────────┼────────────────────■───────────────────┼─────────────────░──╫──╫─┤M├───
           └───┘         └──────┘                                                                                          └───┘   └──────────┘└───┘└─────────┘└───┘   ┌─┴─┐    ┌──────────────┐┌─┴─┐┌──────────────┐┌─┴─┐┌─────────────┐┌─┴─┐┌────────────┐ ░  ║  ║ └╥┘┌─┐
 q74_3: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├────┤ Ry(-0.69352) ├┤ X ├┤ Ry(-0.87728) ├┤ X ├┤ Ry(0.35368) ├┤ X ├┤ Ry(1.2171) ├─░──╫──╫──╫─┤M├
                                                                                                                                                                       └───┘    └──────────────┘└───┘└──────────────┘└───┘└─────────────┘└───┘└────────────┘ ░  ║  ║  ║ └╥┘
meas: 4/════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╩══╩══╩══╩═
                                                                                                                                                                                                                                                                0  1  2  3

In [104]:
def HHL_2(diag: float, offdiag: float, theta: float, t: float):

    NUM_QUBITS = 4  # Total number of qubits
    nb = 1  # Number of qubits representing the solution
    nl = 2  # Number of qubits representing the eigenvalues

    a = diag  # Matrix diagonal
    b = offdiag  # Matrix off-diagonal

    # Initialize the quantum and classical registers
    qr = QuantumRegister(NUM_QUBITS)

    # Create a Quantum Circuit
    qc = QuantumCircuit(qr)

    qrb = qr[0:nb]
    qrl = qr[nb:nb+nl]
    qra = qr[nb+nl:nb+nl+1]

    # State preparation.
    qc.ry(2*theta, qrb[0])

    # QPE with e^{iAt}
    for qu in qrl:
        qc.h(qu)

    qc.p(a*t, qrl[0])
    qc.p(a*t*2, qrl[1])

    qc.u(b*t, -np.pi/2, np.pi/2, qrb[0])


    # Controlled e^{iAt} on \lambda_{1}:
    params=b*t

    qc.p(np.pi/2,qrb[0])
    qc.cx(qrl[0],qrb[0])
    qc.ry(params,qrb[0])
    qc.cx(qrl[0],qrb[0])
    qc.ry(-params,qrb[0])
    qc.p(3*np.pi/2,qrb[0])

    # Controlled e^{2iAt} on \lambda_{2}:
    params = b*t*2

    qc.p(np.pi/2,qrb[0])
    qc.cx(qrl[1],qrb[0])
    qc.ry(params,qrb[0])
    qc.cx(qrl[1],qrb[0])
    qc.ry(-params,qrb[0])
    qc.p(3*np.pi/2,qrb[0])

    # Inverse QFT
    qc.h(qrl[1])
    qc.rz(-np.pi/4,qrl[1])
    qc.cx(qrl[0],qrl[1])
    qc.rz(np.pi/4,qrl[1])
    qc.cx(qrl[0],qrl[1])
    qc.rz(-np.pi/4,qrl[0])
    qc.h(qrl[0])

    # Eigenvalue rotation
    t1=(-np.pi +np.pi/3 - 2*np.arcsin(1/3))/4
    t2=(-np.pi -np.pi/3 + 2*np.arcsin(1/3))/4
    t3=(np.pi -np.pi/3 - 2*np.arcsin(1/3))/4
    t4=(np.pi +np.pi/3 + 2*np.arcsin(1/3))/4

    qc.cx(qrl[1],qra[0])
    qc.ry(t1,qra[0])
    qc.cx(qrl[0],qra[0])
    qc.ry(t2,qra[0])
    qc.cx(qrl[1],qra[0])
    qc.ry(t3,qra[0])
    qc.cx(qrl[0],qra[0])
    qc.ry(t4,qra[0])
    qc.measure_all()

    return qc

    # print(f"Depth: {qc.depth()}")
    # print(f"CNOTS: {qc.count_ops()['cx']}")
    # qc.draw(fold=-1)

In [105]:
qc = HHL_2(1.5, .5, np.pi/4, 1)

In [106]:
from qiskit import IBMQ, transpile


In [107]:
simulator = QasmSimulator()
layout = [2,3,0,4]

compiled_circuit = transpile(qc, backend=simulator, initial_layout=layout)
job = simulator.run(compiled_circuit, shots=1000)

In [108]:
result=job.result()
counts=result.get_counts()
counts

{'1011': 13,
 '1010': 18,
 '1100': 386,
 '1111': 1,
 '0111': 18,
 '0011': 41,
 '0010': 43,
 '0000': 20,
 '0001': 25,
 '0110': 22,
 '1101': 413}

In [109]:
# for mitagating readout errors
from qiskit.utils.mitigation import complete_meas_cal
chip_qubits = 5


meas_cals, state_labels = complete_meas_cal(qubit_list=layout,
                                            qr=QuantumRegister(chip_qubits))
qcs = meas_cals + [compiled_circuit]

job = simulator.run(qcs, shots=1000)

In [110]:
result=job.result()
counts=result.get_counts()
counts[-1]

{'1010': 14,
 '1011': 22,
 '1110': 1,
 '1100': 373,
 '0111': 25,
 '0001': 24,
 '0011': 37,
 '0010': 50,
 '0000': 33,
 '0110': 23,
 '1111': 3,
 '1101': 395}

In [154]:
def extract_counts(dict):
    total_counts = 0
    relevant_count0 = 0
    relevant_count1 = 0
    for key in dict:
        if key[0] == '1':
            if key[-1] == '1':
                relevant_count1 += dict[key]
            if key[-1] == "0":
                relevant_count0 += dict[key]
        total_counts += dict[key]
    return (relevant_count0/total_counts, relevant_count1/total_counts)

In [134]:
p = extract_counts(counts[-1])
p

(0.388, 0.42)

In [135]:
def extract_solution(probs):
    return (np.sqrt(probs[0]), np.sqrt(probs[1]))

In [136]:
sol = extract_solution(p)
sol

(0.6228964600958975, 0.648074069840786)

In [137]:
# calculate fidelity of results
v1 = [0.354, 0.354]
v2 = [np.sqrt(0.440), np.sqrt(0.387)]

np.dot(np.array(v1)/np.linalg.norm(v1), np.array(v2)/np.linalg.norm(v2))

0.9994859457079409

In [138]:
def calculate_normalized_fidelity(v1, v2):
    return np.dot(np.array(v1)/np.linalg.norm(v1), np.array(v2)/np.linalg.norm(v2))

In [139]:
calculate_normalized_fidelity(p, sol)

0.9998044590384589

In [140]:

# load IBM account
IBMQ.save_account('2bb24b1ffb16645433661cd2214aedc8d53fef6a635ac74857b0e282de4e4597754228444daece952006fb5dff87b434835547e926a5539b24f800818c08e394',overwrite=True)
IBMQ.load_account()

provider = IBMQ.providers()
provider = IBMQ.get_provider(hub='ibm-q-education', group='harvard', project='qse-210')
backend=provider.get_backend("ibmq_belem")

ibmqfactory.load_account:WARNING:2023-04-22 15:32:22,836: Credentials are already in use. The existing account in the session will be replaced.


In [144]:
qc2 = HHL_2(1.5, 0.5, -2.23, 1)
layout = [2,3,0,4]

compiled_circuit = transpile(qc2, backend=simulator, initial_layout=layout)
chip_qubits = 5
meas_cals, state_labels = complete_meas_cal(qubit_list=layout,
                                            qr=QuantumRegister(chip_qubits))
qcs2 = meas_cals + [compiled_circuit]
job2 = simulator.run(qcs2, shots=1024)

result2=job2.result()
counts2=result2.get_counts()
p2 = extract_counts(counts2[-1])
sol2 = extract_solution(p2)

classical_sol = (0.555, 0.784)

f2 = calculate_normalized_fidelity(classical_sol, sol2)

print(p2, sol2, f2)
print(counts2[-1])

(0.1171875, 0.076171875) (0.3423265984407288, 0.27599252707274524) 0.9620831217219658
{'1011': 7, '1010': 18, '0110': 23, '0011': 38, '0001': 25, '1110': 4, '1100': 344, '0000': 43, '0010': 54, '0111': 15, '1111': 1, '1101': 452}


In [145]:
calculate_normalized_fidelity(classical_sol, (np.sqrt(0.098), np.sqrt(0.684)))


0.9678739423742864

In [146]:
from qiskit.utils.mitigation import complete_meas_cal

# provider = IBMQ.load_account()

# backend = provider.get_backend('ibmq_quito') # calibrate using real hardware
layout = [2,3,0,4]
chip_qubits = 5

# Transpiled circuit for the real hardware
qc_qa_cx = transpile(qc, backend=backend, initial_layout=layout)

In [147]:
meas_cals, state_labels = complete_meas_cal(qubit_list=layout,
                                            qr=QuantumRegister(chip_qubits))
qcs = meas_cals + [qc_qa_cx]

job = backend.run(qcs, shots=10)

In [177]:
qc2 = HHL_2(1.5, 0.5, -0.57, 1)
layout = [2,3,0,4]

compiled_circuit = transpile(qc2, backend=simulator, initial_layout=layout)
chip_qubits = 5
meas_cals, state_labels = complete_meas_cal(qubit_list=layout,
                                            qr=QuantumRegister(chip_qubits))
qcs2 = meas_cals + [compiled_circuit]
job2 = simulator.run(qcs2, shots=1024)

result2=job2.result()
counts2=result2.get_counts()
p2 = extract_counts(counts2[-1])
sol2 = extract_solution(p2)

classical_sol = (0.790, 0.451)

f2 = calculate_normalized_fidelity(classical_sol, sol2)

print(p2, sol2, f2)
print(counts2[-1])

(0.3955078125, 0.248046875) (0.6288941186718159, 0.49804304532841337) 0.9886111245224527
{'1010': 3, '1011': 11, '0010': 22, '0000': 116, '1110': 3, '1100': 399, '0111': 29, '0001': 133, '0011': 48, '1101': 238, '1111': 5, '0110': 17}
